# Exploracion datos

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb
import ydata_profiling 

In [ ]:
df = pd.read_csv('.\\data/datos_apartamentos_rent.csv',sep=';')


In [ ]:
ydata_profiling.ProfileReport(df)